In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### 데이터 불러오기

In [89]:
df = pd.read_csv('weather_2124.csv', encoding= 'cp949')
df = df.rename(columns = {'평균기온(°C)' : '평균기온',
                          '최저기온(°C)' : '최저기온',
                          '최고기온(°C)' : '최고기온', 
                          '일강수량(mm)' : '일강수량',
                          '평균 풍속(m/s)' : '풍속',
                          '평균 상대습도(%)' : '습도'})

df_seoul = df[df['지점명'] == '서울']

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11820 entries, 0 to 11819
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지점      11820 non-null  int64  
 1   지점명     11820 non-null  object 
 2   일시      11820 non-null  object 
 3   평균기온    11820 non-null  float64
 4   최저기온    11818 non-null  float64
 5   최고기온    11819 non-null  float64
 6   일강수량    4563 non-null   float64
 7   풍속      11809 non-null  float64
 8   습도      11815 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 831.2+ KB


In [91]:
df.isnull().sum()

지점         0
지점명        0
일시         0
평균기온       0
최저기온       2
최고기온       1
일강수량    7257
풍속        11
습도         5
dtype: int64

### 전처리 1. 강수량

In [92]:
# 1-1 강수량 null 은 0으로 처리 
df['일강수량'] = df['일강수량'].fillna(0)


# 1-2 강수여부 컬럼 추가 
df['강수여부'] = np.where(df[['일강수량']] == 0, 'sunny', 'rainy')
df

,지점,지점명,일시,평균기온,최저기온,최고기온,일강수량,풍속,습도,강수여부
0,108,서울,2021-01-01,-4.2,-9.8,1.6,0.0,2.0,64.0,sunny
1,108,서울,2021-01-02,-5.0,-8.4,-1.4,0.0,2.6,38.5,sunny
2,108,서울,2021-01-03,-5.6,-9.1,-2.0,0.0,2.0,45.0,sunny
3,108,서울,2021-01-04,-3.5,-8.4,0.3,0.0,1.7,51.4,sunny
4,108,서울,2021-01-05,-5.5,-9.9,-2.1,0.0,2.9,52.8,sunny
...,...,...,...,...,...,...,...,...,...,...
11815,159,부산,2024-03-23,16.1,12.3,20.5,0.7,5.2,73.6,rainy
11816,159,부산,2024-03-24,11.3,10.1,13.6,3.7,4.2,77.5,rainy
11817,159,부산,2024-03-25,10.5,8.7,12.4,9.4,4.2,82.4,rainy
11818,159,부산,2024-03-26,8.8,6.2,12.9,9.8,3.8,75.4,rainy


### 전처리 2. 풍속, 습도 null값 전처리

 - 각 지역 별 평균 값으로 입력

In [93]:
grouped = df['풍속'].groupby(df['지점명']).mean()
grouped

지점명
광주    1.310932
대구    2.237648
대전    1.668528
부산    3.202462
서울    2.332965
수원    1.908122
울산    2.141794
인천    2.800677
창원    1.452030
포항    2.752030
Name: 풍속, dtype: float64

In [94]:
df['풍속'] = df['풍속'].fillna(df.groupby('지점명')['풍속'].transform('mean'))
df['습도'] = df['습도'].fillna(df.groupby('지점명')['습도'].transform('mean'))

### 전처리3. 최고기온 최저기온 null값 처리

- 바로 전 날의 데이터로 결측치 대체

In [95]:
df['최고기온'] = df['최고기온'].fillna(method = 'pad')
df['최저기온'] = df['최저기온'].fillna(method = 'pad')

/var/folders/_r/cptctqz510x3xvwm2fmrfz_r0000gn/T/ipykernel_46129/3093923572.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['최고기온'] = df['최고기온'].fillna(method = 'pad')
/var/folders/_r/cptctqz510x3xvwm2fmrfz_r0000gn/T/ipykernel_46129/3093923572.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['최저기온'] = df['최저기온'].fillna(method = 'pad')


In [97]:
df.isnull().sum()

지점      0
지점명     0
일시      0
평균기온    0
최저기온    0
최고기온    0
일강수량    0
풍속      0
습도      0
강수여부    0
dtype: int64